In [3]:
import time
import os
import rasterio
import numpy as np
from rasterio.merge import merge
import requests
import time
import csv
import pandas as pd
import os
import geopandas as gpd
from rasterio.mask import mask
from shapely.geometry import box, Point
from pyproj import CRS, Transformer

# Mapbox Elevation API on non-LiDAR coords

In [21]:
import requests
import pandas as pd
import time

# Your Google Maps API Key
GOOGLE_API_KEY = "API_key"

# Function to query elevation for multiple locations
def get_elevations(locations):
    url = "https://maps.googleapis.com/maps/api/elevation/json"
    params = {
        'locations': '|'.join(locations),  # Join locations with '|'
        'key': GOOGLE_API_KEY
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            return [result['elevation'] for result in data['results']]
    return [None] * len(locations)  # Return None for all locations if the request fails

# Read the input CSV file
def process_csv(input_file, output_file, batch_size=512):
    # Load CSV file into a pandas DataFrame
    df = pd.read_csv(input_file)

    # Check for required columns
    if 'Latitude' not in df.columns or 'Longitude' not in df.columns:
        raise ValueError("CSV file must contain 'Latitude' and 'Longitude' columns")

    # Prepare locations in the required format
    df['location'] = df.apply(lambda row: f"{row['Latitude']},{row['Longitude']}", axis=1)

    # Initialize a list to store elevations
    elevations = []

    # Process in batches
    for i in range(0, len(df), batch_size):
        batch_locations = df['location'][i:i + batch_size].tolist()
        batch_elevations = get_elevations(batch_locations)
        elevations.extend(batch_elevations)
        print(f"Processed {i + len(batch_locations)} of {len(df)} locations")
        time.sleep(1)  # Add a delay to respect rate limits

    # Add the elevations to the DataFrame
    df['elevation'] = elevations

    # Save the updated DataFrame to a new CSV file
    df.to_csv(output_file, index=False)
    print(f"Elevations added and saved to {output_file}")

# Main execution block
if __name__ == "__main__":
    input_csv = "Book1.csv"  # Replace with your input CSV file
    output_csv = "Book1_elevations.csv"  # Replace with your desired output file
    try:
        process_csv(input_csv, output_csv)
    except Exception as e:
        print(f"An error occurred: {e}")

Processed 11 of 11 locations
Elevations added and saved to Book1_elevations.csv


# Find coordinate elevation using LINZ 1m DEM

In [55]:
def get_elevation_at_coordinates(tiff_folder, coords):
    """
    Finds the elevation of the coordinates within the DEMs in the provided folder.

    :param tiff_folder: Path to the folder containing .tiff DEM files
    :param coords: List of tuples containing coordinates [(longitude, latitude), ...]
    :return: A list of elevations corresponding to the provided coordinates
    """
    # Convert coordinates to a GeoDataFrame
    coords_gdf = gpd.GeoDataFrame(geometry=[Point(lon, lat) for lon, lat in coords], crs="EPSG:2193")  # NZTM CRS

    # Step 1: Build spatial index for DEM files
    dem_files = []
    file_count = 0
    num_files = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
    for filename in os.listdir(tiff_folder):
        file_count += 1
        if file_count % 500 ==0:
            print(f"{100*file_count/num_files}% files processed")
        if filename.endswith('.tif'):
            #print(filename)
            tiff_path = os.path.join(tiff_folder, filename)
            with rasterio.open(tiff_path) as src:
                bounds = src.bounds
                dem_files.append({
                    'filename': tiff_path,
                    'geometry': box(bounds.left, bounds.bottom, bounds.right, bounds.top),
                    'crs': src.crs
                })

    # Convert DEM metadata to a GeoDataFrame
    dem_gdf = gpd.GeoDataFrame(dem_files, crs="EPSG:2193")  # Match NZTM CRS

    # Step 2: Spatial join to pre-filter DEMs for each coordinate
    relevant_dems = gpd.sjoin(coords_gdf, dem_gdf, how='inner', predicate='intersects')

    # Step 3: Initialize results
    elevations_dict = {coord: -99.0 for coord in coords}  # Default to -99.0 for all coordinates

    # Step 4: Process each DEM
    for _, dem in dem_gdf.iterrows():
        dem_coords = relevant_dems[relevant_dems['filename'] == dem['filename']]
        if dem_coords.empty:
            continue  # Skip if no coordinates fall within this DEM

        # Load DEM data and affine transform
        try:
            with rasterio.open(dem['filename']) as src:
                dem_data = src.read(1)  # Read elevation data
                transform = src.transform  # Affine transform

                # Convert coordinates to pixel indices
                rows, cols = zip(*[src.index(pt.x, pt.y) for pt in dem_coords.geometry])
                rows, cols = np.array(rows), np.array(cols)

                # Extract elevations
                dem_elevations = dem_data[rows, cols]
                for idx, elevation in enumerate(dem_elevations):
                    point = dem_coords.geometry.iloc[idx]
                    coord = (point.x, point.y)

                    if elevation != -9999.0:  # Ignore NoData values
                        elevations_dict[coord] = elevation

        except Exception as e:
            print(f"Error processing {dem['filename']}: {e}")

    # Convert the dictionary to a list of elevations in the original input order
    elevations = [elevations_dict[coord] for coord in coords]

    return elevations

# Path to the folder with your .tiff DEM files

tiff_folder = 'DEMs_Cheviot'

# Assuming coords is a DataFrame with 'east_NZTM' and 'north_NZTM' columns
coordinates = list(zip(coords['east_NZTM'], coords['north_NZTM']))  # [(lon, lat), ...]

elevations = get_elevation_at_coordinates(tiff_folder, coordinates)

500 files processed
1000 files processed
1500 files processed
2000 files processed
2500 files processed
3000 files processed
3500 files processed
4000 files processed
4500 files processed
5000 files processed
5500 files processed
6000 files processed
6500 files processed
7000 files processed
7500 files processed
8000 files processed
8500 files processed
9000 files processed
9500 files processed
10000 files processed
10500 files processed
11000 files processed
11500 files processed
12000 files processed
12500 files processed
13000 files processed
13500 files processed
14000 files processed
14500 files processed
15000 files processed
15500 files processed
16000 files processed
16500 files processed
17000 files processed
17500 files processed
18000 files processed
18500 files processed
19000 files processed
19500 files processed
20000 files processed
20500 files processed
21000 files processed
21500 files processed
22000 files processed
22500 files processed
23000 files processed
23500 fi

In [56]:
def append_elevation_to_dataframe(tiff_folder, coords_df, lon_col, lat_col):
    """
    Appends elevation data to a DataFrame based on the coordinates.

    :param tiff_folder: Path to the folder containing .tiff DEM files
    :param coords_df: DataFrame with longitude and latitude columns
    :param lon_col: Name of the longitude column in the DataFrame
    :param lat_col: Name of the latitude column in the DataFrame
    :return: Updated DataFrame with a new 'elevation' column
    """
    # Extract coordinates from the DataFrame
    coords = list(zip(coords_df[lon_col], coords_df[lat_col]))

    # Get elevations for the coordinates
    elevations = get_elevation_at_coordinates(tiff_folder, coords)

    # Append the elevations to the DataFrame
    coords_df['Elevation (m)1'] = elevations

    return coords_df


# Example usage
start = time.time()

# Path to the folder with your .tiff DEM files

tiff_folder = 'DEMs_Cheviot'

# Assuming coords is a DataFrame with 'east_NZTM' (longitude) and 'north_NZTM' (latitude) columns
lon_col, lat_col = 'east_NZTM', 'north_NZTM'

# Append elevation data
coords_with_elevation = append_elevation_to_dataframe(tiff_folder, coords, lon_col, lat_col)

#print(coords_with_elevation)  # Display the first few rows
print(f"Processing time: {time.time() - start:.2f} seconds")

500 files processed
1000 files processed
1500 files processed
2000 files processed
2500 files processed
3000 files processed
3500 files processed
4000 files processed
4500 files processed
5000 files processed
5500 files processed
6000 files processed
6500 files processed
7000 files processed
7500 files processed
8000 files processed
8500 files processed
9000 files processed
9500 files processed
10000 files processed
10500 files processed
11000 files processed
11500 files processed
12000 files processed
12500 files processed
13000 files processed
13500 files processed
14000 files processed
14500 files processed
15000 files processed
15500 files processed
16000 files processed
16500 files processed
17000 files processed
17500 files processed
18000 files processed
18500 files processed
19000 files processed
19500 files processed
20000 files processed
20500 files processed
21000 files processed
21500 files processed
22000 files processed
22500 files processed
23000 files processed
23500 fi

In [61]:
print(coords_with_elevation[coords_with_elevation['Elevation (m)1']!=-99].copy())
eles = coords_with_elevation[coords_with_elevation['Elevation (m)1']!=-99].copy()
#eles.to_csv(f"Vehicle_data/Elevation_INSERT_AREA_NAME_HERE.csv")
c = coords_with_elevation[coords_with_elevation['Elevation (m)1']==-99].copy()
#c.to_csv(f"Vehicle_data/No_elevationINSERTNUMBERHERE.csv")

      Unnamed: 0.5  Unnamed: 0.4  Unnamed: 0.3  Unnamed: 0.2  Unnamed: 0.1  \
2350          2499          2825          2825          4685          5008   
2351          2500          2826          2826          4686          5009   
2352          2501          2827          2827          4687          5010   
2353          2502          2828          2828          4688          5011   
2354          2503          2829          2829          4689          5012   
...            ...           ...           ...           ...           ...   
3147          3296          3622          3622          6144          6467   
3148          3297          3623          3623          6145          6468   
3149          3298          3624          3624          6146          6469   
3150          3299          3625          3625          6147          6470   
3151          3300          3626          3626          6148          6471   

      Unnamed: 0            Vehicle IGN Event Name  Longitude  

# Find coordinates not covered by DEM

In [51]:
coordinates = list(zip(coords['east_NZTM'], coords['north_NZTM']))

In [49]:
start = time.time()
wgs84 = CRS.from_epsg(4326)  # WGS84 (EPSG:4326)
nztm2000 = CRS.from_epsg(2193)  # NZTM2000 (EPSG:2193)

# Create a Transformer object for converting WGS84 to NZTM2000
transformer = Transformer.from_crs(wgs84, nztm2000, always_xy=True)

def wgs84_to_nztm2000(lat, lon):
    """
    Converts WGS 84 coordinates (latitude, longitude) to NZTM2000.

    Parameters:
        lat (float): Latitude in WGS84
        lon (float): Longitude in WGS84

    Returns:
        tuple: (eastings, northings) in NZTM2000
    """
    # Transform the coordinates from WGS84 to NZTM2000
    easting, northing = transformer.transform(lon, lat)
    coords['north_NZTM'] = northing
    coords['east_NZTM'] = easting
    
    return easting, northing

# Example coordinates in WGS84
lat_wgs84 = coords['Latitude']# Latitude of Auckland in WGS84
lon_wgs84 = coords['Longitude']   # Longitude of Auckland in WGS84

# Convert to NZTM2000
easting, northing = wgs84_to_nztm2000(lat_wgs84, lon_wgs84)

#print(coords)
end = time.time()
#print(f'{end-start:.2f}')

In [19]:
#coords = pd.read_csv(f"GPS_data")
print(coords)

    Unnamed: 0   Latitude   Longitude
0            0 -41.658920  173.655256
1            1 -41.660656  173.655428
2            2 -41.661138  173.656265
3            3 -41.661026  173.656806
4            4 -41.658143  173.654256
5            5 -41.658720  173.655173
6            6 -41.660220  173.655458
7            7 -41.660591  173.655336
8            8 -41.661111  173.656033
9            9 -41.661163  173.656324
10          10 -41.660920  173.656915
